In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 995 kB 6.1 MB/s 
     |████████████████████████████████| 384 kB 47.5 MB/s 
     |████████████████████████████████| 140 kB 42.5 MB/s 
     |████████████████████████████████| 58 kB 5.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.
Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.

In [2]:

# 라이브러리 임포트
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium.webdriver.common.by import By
from urllib.parse import quote
import warnings
warnings.filterwarnings('ignore')

# 기본 세팅
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)    # 드라이버 선언, 드라이버 들어가기

# 데이터 저장용 데이터 프래임
movie_ost_name = pd.DataFrame(columns = ['movie_name', 'ost_name', 'artist_name', 'year'])

# 크롤링할 사이트(벅스) url 가져오고 링크 열기
url = "https://music.bugs.co.kr/years/movieost"
driver.get(url)
driver.implicitly_wait(3)
time.sleep(1.5)

# html 파일 가져오기
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')

# 년도별 OST 콜랙션 제목 타게팅
ul = soup.find('ul', {'class', 'list'})
li = ul.find_all('li') 

# 순서대로 영화 개봉 년도 들어가기 및 크롤링하기

# ***중요*** 크롤링 하는 양이 너무 많아서 한번에 안돌아갑니다.(오류남) 실제로 돌리려면 아래 range를 1-9, 9-18, 18-27로 나눠서 돌리시는걸 추천합니다.
# 오류 수정 불가한 이유 : 크롤링하는 환경이 내 pc(로컬)가 아닌 구글 드라이브의 어딘가이기 때문에 오류가 나는 명확한 이유를 찾기 힘들다.

for i in range(1, 27): 
#  a = li[i].find('a', {'class', 'title hyrend'})
  driver.find_element(By.XPATH, '//*[@id="container"]/section/div/ul/li[{}]/figure/figcaption/a[1]'.format(i+1)).click()
  time.sleep(2)

  # 새로 띄운 영화 창 주소 가져와서 다시 입력
  driver.get(driver.current_url)
  driver.implicitly_wait(3)
  print(driver.current_url)

  html_source = driver.page_source
  soup = BeautifulSoup(html_source, 'html.parser')

  # 년도 정보 크롤링
  year = driver.find_element(By.XPATH, '//*[@id="container"]/header/div/h1').text
  year = year[0:4]

  # 영화제목, ost 제목등이 있는 tr 라벨 선택
  table = soup.find_all('table', {'class', 'list'})
  tr = table[0].find_all('tr')

  for index_l in range(0, len(tr)):
    try:
      # 영화 ost 제목
      o_name = tr[index_l].find('p').find('a').text.strip()
      time.sleep(2)

      # 영화 제목
      m_name = tr[index_l].find_all('td')[4].text.strip()
      time.sleep(2)

      if 'a' <= m_name[0] <= "z" or 'A' <= m_name[0] <='Z':
        m_name = m_name.split('(')[0].strip()
        time.sleep(2)

      else:
        if 'a' <= m_name.split('(')[1][0] <= "z" or 'A' <= m_name.split('(')[1][0] <='Z':
          m_name = m_name.split('(')[1]
          m_name = m_name.split(')')[0].strip()
        else:
          m_name = m_name.split('(')[2]
          m_name = m_name.split(')')[0].strip()
        time.sleep(2)

      # 아티스트 정보
      artist = tr[index_l].find_all('td')[3].text.strip()
      time.sleep(2)

      if '(' in artist:
        artist = artist.split('(')[0]

      # 영화 정보 저장
      movie_ost_name = movie_ost_name.append({'movie_name' : m_name, 'ost_name' : o_name, 'artist_name' : artist, 'year' : year}, ignore_index=True)
      print('영화 제목 : ' +m_name + '  /  ost제목 : ' + o_name + '  /  아티스트 이름 : ' + artist + '  /  개봉년도' + year)
      time.sleep(1.5)

    except:
      time.sleep(2)
      print('\n')
      print('버릴 데이터')
      print(m_name)
      print('\n')
      pass
  print('초기 화면으로')
  time.sleep(2)

movie_ost_name.to_csv('영화ost크롤링.csv', encoding = 'utf-8-sig', index = False) 

https://music.bugs.co.kr/musicpd/albumview/44024
영화 제목 : Bad Boys For Life  /  ost제목 : RITMO (Bad Boys For Life)  /  아티스트 이름 : Black Eyed Peas  /  개봉년도2020
영화 제목 : Bad Boys For Life  /  ost제목 : Muévelo  /  아티스트 이름 : Nicky Jam  /  개봉년도2020


버릴 데이터
나쁜 녀석들 : 포에버 (Bad Boys For Life) OST




KeyboardInterrupt: ignored

In [ ]:
import pandas as pd

df1 = pd.read_csv('/content/완성코드_750.csv')
df2 = pd.read_csv('/content/완성코드_750_1500.csv')
df3 = pd.read_csv('/content/완성코드1500~.csv')


result = pd.concat([df1, df2], ignore_index=True)

result = pd.concat([result, df3], ignore_index=True)

result.to_csv('영화ost크롤링.csv', encoding = 'utf-8-sig', index = False) 

In [ ]:
result.head(-10)

movie_name  \
0         Spider-Man: No Way Home   
1         Spider-Man: No Way Home   
2         Spider-Man: No Way Home   
3               Last One Standing   
4     Venom: Let There Be Carnage   
...                           ...   
2346       My Way [Clean Version]   
2347            Spawn - The Album   
2348            Spawn - The Album   
2349            Spawn - The Album   
2350                Feels So Good   

                                               ost_name        artist_name  \
0     Arachnoverture (from "Spider-Man: No Way Home"...  Michael Giacchino   
1     Forget Me Knots (from "Spider-Man: No Way Home...  Michael Giacchino   
2     Peter Parker Picked a Perilously Precarious Pr...  Michael Giacchino   
3     Last One Standing (From Venom: Let There Be Ca...        Skylar Grey   
4                    St. Estes Reform School (Extended)     Marco Beltrami   
...                                                 ...                ...   
2346                            Slow Jam (feat. Monica)              Usher   
2347                                       One Man Army            Prodigy   
2348                         (Can't You) Trip Like I Do     Crystal Method   
2349      For Whom The Bell Tolls (The Irony Of It All)          Metallica   
2350                         The Tide Is High (Rremake)        Maxi Priest   

      year          장르      감독          movie_name_low  valence  acousticness  \
0     2021  액션,어드벤처,SF    존 왓츠      spidermannowayhome   0.0537       0.45300   
1     2021  액션,어드벤처,SF    존 왓츠      spidermannowayhome   0.0300       0.85000   
2     2021  액션,어드벤처,SF    존 왓츠      spidermannowayhome   0.0672       0.72900   
3     2021           0       0         lastonestanding   0.2330       0.23700   
4     2021   액션,SF,스릴러  앤디 서키스  venomlettherebecarnage   0.0391       0.11800   
...    ...         ...     ...                     ...      ...           ...   
2346  1997           0       0       mywaycleanversion   0.1970       0.01540   
2347  1997           0       0           spawnthealbum      NaN           NaN   
2348  1997           0       0           spawnthealbum      NaN           NaN   
2349  1997           0       0           spawnthealbum   0.3730       0.00438   
2350  1997           0       0             feelssogood      NaN           NaN   

      danceability  energy  loudness    tempo  
0           0.1760  0.2010   -15.155  136.426  
1           0.0691  0.0614   -19.004   73.215  
2           0.2910  0.0304   -16.878   60.185  
3           0.6640  0.6920    -4.128  154.924  
4           0.1850  0.2340   -15.574  113.130  
...            ...     ...       ...      ...  
2346        0.7070  0.4470    -8.131  127.968  
2347           NaN     NaN       NaN      NaN  
2348           NaN     NaN       NaN      NaN  
2349        0.5290  0.9730    -7.709  160.025  
2350           NaN     NaN       NaN      NaN  

[2351 rows x 13 columns]